In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

In [2]:
df=pd.read_csv('./train.csv')
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df=df[['Pclass','Name','Sex','Age','SibSp','Parch','Fare','Embarked','Survived']]

In [4]:
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,0
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,1
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,1
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,1
4,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,0


# Data Cleaning

In [5]:
for col in df.columns.values:
    print(col,'--',df[col].isnull().sum())

Pclass -- 0
Name -- 0
Sex -- 0
Age -- 177
SibSp -- 0
Parch -- 0
Fare -- 0
Embarked -- 2
Survived -- 0


In [6]:
df['Age']=df['Age'].fillna(df['Age'].dropna().median())

In [7]:
most_frequent=df['Embarked'].dropna().mode()[0]
df['Embarked']=df['Embarked'].fillna(most_frequent)

In [8]:
for col in df.columns.values:
    print(col,'--',df[col].isnull().sum())

Pclass -- 0
Name -- 0
Sex -- 0
Age -- 0
SibSp -- 0
Parch -- 0
Fare -- 0
Embarked -- 0
Survived -- 0


# Data Transformation

In [9]:
df.head(2)

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,0
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,1


In [10]:
sex_mapping={'male':0,'female':1}
embarked_mapping={'S':0,'C':1,'Q':2}
df['Sex']=df['Sex'].map(sex_mapping)
df['Embarked']=df['Embarked'].map(embarked_mapping)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,7.2500,0,0
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,71.2833,1,1
2,3,"Heikkinen, Miss. Laina",1,26.0,0,0,7.9250,0,1
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,53.1000,0,1
4,3,"Allen, Mr. William Henry",0,35.0,0,0,8.0500,0,0


In [11]:
import re
for i in range(len(df)):
    if(df['Sex'][i]):
        df['Name'][i]=2
    else:
        if(len(re.findall('Master',df['Name'][i]))!=0):
            df['Name'][i]=1
        else:
            df['Name'][i]=0
df.head()


E:\anaconda\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
E:\anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
E:\anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,3,0,0,22.0,1,0,7.2500,0,0
1,1,2,1,38.0,1,0,71.2833,1,1
2,3,2,1,26.0,0,0,7.9250,0,1
3,1,2,1,35.0,1,0,53.1000,0,1
4,3,0,0,35.0,0,0,8.0500,0,0


In [12]:
X=df.drop(['Survived'],axis=1).values
y=df['Survived'].values
print(X.shape,y.shape)

(891, 8) (891,)


# Training Classifiers

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(623, 8) (623,) (268, 8) (268,)


## Decision Tree 

In [14]:
clf1=tree.DecisionTreeClassifier()
clf1=clf1.fit(X_train,y_train)
score1=clf1.score(X_test,y_test)
print(score1)

0.8097014925373134


## SVM Polynomial

In [15]:
#svm = SVC(kernel='poly')
#svm.fit(X_train,y_train)
# score2=svm.score(X_test,y_test)
# print(score2)

## SVM RBF and Linear

In [16]:
svm_rbf = SVC(kernel='rbf')
svm_rbf.fit(X_train,y_train)
score3=svm_rbf.score(X_test,y_test)
print(score3)
svm_len= SVC(kernel='linear')
svm_len.fit(X_train,y_train)
score4=svm_len.score(X_test,y_test)
print(score4)

0.75
0.8097014925373134


### Thus the SVM Linear and Decision Tree Perform the best

## Gaussian Naive Bayes

In [17]:
clf2=GaussianNB()
clf2.fit(X_train,y_train)
score5=clf2.score(X_test,y_test)
print(score5)

0.7873134328358209


## KNN 5 Neighbors

In [18]:
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train,y_train)
print(knn.score(X_test,y_test))

0.7574626865671642


In [19]:
clf=tree.DecisionTreeClassifier()
clf.fit(X,y)
svc=SVC(kernel='linear')
svc.fit(X,y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

# Test Data Cleaning

In [20]:
test=pd.read_csv('./test.csv')
print(test.columns)

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [22]:
test=test[['PassengerId','Pclass','Name','Sex','Age','SibSp','Parch','Fare','Embarked']]
for col in test.columns.values:
    print(col,'--',test[col].isnull().sum())

PassengerId -- 0
Pclass -- 0
Name -- 0
Sex -- 0
Age -- 86
SibSp -- 0
Parch -- 0
Fare -- 1
Embarked -- 0


In [23]:
test['Age']=test['Age'].fillna(test['Age'].dropna().median())
test['Fare']=test['Fare'].fillna(test['Fare'].dropna().median())

E:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
E:\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [24]:
for col in test.columns.values:
    print(col,'--',test[col].isnull().sum())

PassengerId -- 0
Pclass -- 0
Name -- 0
Sex -- 0
Age -- 0
SibSp -- 0
Parch -- 0
Fare -- 0
Embarked -- 0


In [25]:
test['Sex']=test['Sex'].map(sex_mapping)
test['Embarked']=test['Embarked'].map(embarked_mapping)

E:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
E:\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [26]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,"Kelly, Mr. James",0,34.5,0,0,7.8292,2
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.0,1,0,7.0000,0
2,894,2,"Myles, Mr. Thomas Francis",0,62.0,0,0,9.6875,2
3,895,3,"Wirz, Mr. Albert",0,27.0,0,0,8.6625,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.0,1,1,12.2875,0


In [27]:
for i in range(len(test)):
    if(test['Sex'][i]):
        test['Name'][i]=2
    else:
        if(len(re.findall('Master',test['Name'][i]))!=0):
            test['Name'][i]=1
        else:
            test['Name'][i]=0
test.head()

E:\anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
E:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:2910: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
E:\anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
E:\anaconda\lib\site-packages\ipykernel_la

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,0,0,34.5,0,0,7.8292,2
1,893,3,2,1,47.0,1,0,7.0000,0
2,894,2,0,0,62.0,0,0,9.6875,2
3,895,3,0,0,27.0,0,0,8.6625,0
4,896,3,2,1,22.0,1,1,12.2875,0


In [28]:
Xtest=test.drop(['PassengerId'],axis=1)

In [29]:
Xtest.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,0,34.5,0,0,7.8292,2
1,3,2,1,47.0,1,0,7.0000,0
2,2,0,0,62.0,0,0,9.6875,2
3,3,0,0,27.0,0,0,8.6625,0
4,3,2,1,22.0,1,1,12.2875,0


In [31]:
passengerId=test['PassengerId'].values


In [32]:
output1=clf.predict(Xtest)
output2=svc.predict(Xtest)
print(output1.shape,output2.shape,passengerId.shape)

(418,) (418,) (418,)


In [33]:
import csv
with open('output1.csv','w',newline='') as f:
    field=['PassengerId','Survived']
    writer=csv.DictWriter(f,fieldnames=field)
    writer.writeheader()
    for i in range(len(passengerId)):
        writer.writerow({'PassengerId':passengerId[i],'Survived':output1[i]})

with open('output2.csv','w',newline='') as f:
    field=['PassengerId','Survived']
    writer=csv.DictWriter(f,fieldnames=field)
    writer.writeheader()
    for i in range(len(passengerId)):
        writer.writerow({'PassengerId':passengerId[i],'Survived':output2[i]})

# On actual submission on Kaggle SVM_LEN got 0.78 while Decision Tree got 0.68